# 4.4.4 Implementing the Query System

Section placed first so query system can be used in 4.4.1-3 exercises

### Supporting Code from Previous Chapters

#### Table Data Structure from Chapter 3

In [1]:
(define (assoc key records)
  (cond ((null? records) #f)
	((equal? key (caar records)) (car records))
	(else
	    (assoc key (cdr records)))))

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record
                  (cdr record)
                  #f))
            #f)))
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record
                  (set-cdr! record value)
                  (set-cdr! subtable
                            (cons (cons key-2 value)
                                  (cdr subtable)))))
            (set-cdr! local-table
                      (cons (list key-1
                                  (cons key-2 value))
                            (cdr local-table)))))
      'ok)    
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation -- TABLE" m))))
    dispatch))

#### Stream Implementation from Chapter 3

In [2]:
; define as macros to override Schemes eager evalutation
(define-syntax delay
               [(delay ?exp) (memo-proc (lambda() ?exp))])
(define-syntax cons-stream
               [(cons-stream ?a ?b) (cons ?a (delay ?b))])

(define the-empty-stream '())

(define (stream-null? s)
  (eq? s the-empty-stream))
(define (stream-car stream)
  (car stream))
(define (stream-cdr stream)
  (force (cdr stream)))
(define (list->stream lst)
  (accumulate (lambda (a b)
                ;; wrap in lambda as cons-stream is a macro
                ;; -> cannot pass directly as first class function
                (cons-stream a b))
              the-empty-stream lst))

(define (stream-append s1 s2)
  (if (stream-null? s1)
      s2
      (cons-stream
       (stream-car s1)
       (stream-append (stream-cdr s1) s2))))

(define (stream-ref s n)
  (if (= n 0)
      (stream-car s)
      (stream-ref (stream-cdr s) (- n 1))))

(define (stream-map proc s)
  (if (stream-null? s)
      the-empty-stream
      (cons-stream
       (proc (stream-car s))
       (stream-map proc (stream-cdr s)))))

(define (stream-for-each proc s)
  (if (stream-null? s)
      'done
      (begin
       (proc (stream-car s))
       (stream-for-each proc
                        (stream-cdr s)))))

(define (stream-filter pred stream)
  (cond ((stream-null? stream)
         the-empty-stream)
        ((pred (stream-car stream))
         (cons-stream
          (stream-car stream)
          (stream-filter
           pred
           (stream-cdr stream))))
        (else (stream-filter
               pred
               (stream-cdr stream)))))

(define (stream-enumerate-interval low high)
  (if (> low high)
      the-empty-stream
      (cons-stream
       low
       (stream-enumerate-interval (+ low 1)
                                  high))))

(define (force delayed-object)
  (delayed-object))

(define (memo-proc proc)
  (let ((already-run? #f)
        (result #f))
    (lambda ()
      (if (not already-run?)
          (begin (set! result (proc))
                 (set! already-run? #t)
                 result)
          result))))

(define (display-stream s)
  (stream-for-each display-line s))

(define (display-line x)
  (newline)
  (display x))

;; from previous chapter
(define (accumulate op initial sequence)
  (if (null? sequence)
      initial
      (op (car sequence)
          (accumulate op
                      initial
                      (cdr sequence)))))

(define (tagged-list? exp tag)
  (and (pair? exp)
       (eq? (car exp) tag)))

### 4.4.4.1 Driver Loop and Instantiation

In [3]:
(define input-prompt  ";;; Query input:")
(define output-prompt ";;; Query results:")

;; TODO: READ-INPUT doesn't always work
;; i.e. will turn (and () () ) into (if) expression
;; may need to update Calysto Scheme (read) implementation
;; to behave like Scheme https://www.gnu.org/software/mit-scheme/documentation/mit-scheme-ref/Input-Procedures.html
;; Use inter-pret-query instead

;; see 4.4.4.3 Patterns with dotted tails for desired behaviour
;; calysto scheme version of read procedure returns a string
;; whereas scheme read procedure parses the string to a scheme object (pair etc)
;; evaluator is designed to accept scheme objects not raw strings
(define (read-input)
  (unparse ;; turn AST into scheme code
   (parse-string ;; parse to calysto scheme AST
    (read)))) ;; read user input as string

(define (query-driver-loop)
  (prompt-for-input input-prompt)
  (let ((q (query-syntax-process (read-input))))
    ;; expression is rule/assertion to be added to database
    (cond ((assertion-to-be-added? q)
           (add-rule-or-assertion!
            (add-assertion-body q))
           (newline)
           (display
            "Assertion added to data base.")
           (query-driver-loop))
          ;; expression is a query
          (else
           (newline)
           (display output-prompt)
           (display-stream
            (stream-map
             ;; result of evalutation = stream of frames generated
             ;; by satisfying query w/ vars found in database
             ;; used to form new stream of copies of original query w/ instantiated vars
             ;; w/ values from stream of frames from evaluation
             (lambda (frame)
               (instantiate
                q
                frame
                (lambda (v g)
                  (contract-question-mark v))))
             ;; eval query w/ initial frame stream of single empty frame
             (qeval q (singleton-stream '()))))
            (query-driver-loop))))) 

;; instantiate an expression
;; -> copy and replace vars with values in a given frame
;; -> values are also instantiated (could contain variables)
(define (instantiate
         exp frame unbound-var-handler)
  (define (copy exp)
    (cond ((var? exp)
           (let ((binding
                  (binding-in-frame
                   exp frame)))
             (if binding
                 (copy
                  (binding-value binding))
                 (unbound-var-handler
                  exp frame))))
          ((pair? exp)
           (cons (copy (car exp))
                 (copy (cdr exp))))
          (else exp)))
    (copy exp))

;; from Section 4.1
(define (prompt-for-input string)
  (newline) (newline) (display string) (newline))


### 4.4.4.2 The Evaluator

In [4]:
(define (qeval query frame-stream)
  ;; identify special forms w/ data-directed dispatch on query type
  (let ((qproc (get (type query) 'qeval)))
    (if qproc
        (qproc (contents query) frame-stream)
        (simple-query query frame-stream))))

;; returns stream formed by extending each input framee
;; by all databse matches of query
(define (simple-query query-pattern
                      frame-stream)
  ;; form a stream of all ways any frames in original input
  ;; can be extended to produce a match with query-pattern
  (stream-flatmap
   (lambda (frame)
     ;; make stream of all ways query-pattern
     ;; can be satisfied consistent w/ original frame
     (stream-append-delayed
      ;; match pattern against all assertion in database
      ;; -> produce stream of extended frames
      (find-assertions query-pattern frame)
      (delay
       ;; apply all possible rules
       ;; -> produce another stream of extended frames
       (apply-rules query-pattern frame))))
   frame-stream))

;; 'and' compound query
;; -> process frame-stream to find stream of all possible frame
;;    extensions satisfying first query in conjunction
;; -> recursively call conjoin on remaining queries
(define (conjoin conjuncts frame-stream)
  (if (empty-conjunction? conjuncts)
      frame-stream
      ;; process as a series combination of queries
      (conjoin (rest-conjuncts conjuncts)
               ;; use output frame-stream of first conjunct
               ;; as input frame-stream to next conjunct
               (qeval
                (first-conjunct conjuncts)
                frame-stream))))

;; 'or' compound query
;; -> compute possible output streams for each disjunct separately
;; -> return merged output streams
(define (disjoin disjuncts frame-stream)
  (if (empty-disjunction? disjuncts)
      the-empty-stream
      (interleave-delayed
       (qeval (first-disjunct disjuncts)
              frame-stream)
       (delay (disjoin
               (rest-disjuncts disjuncts)
               frame-stream)))))

;; 'not' compound query
;; -> attempt to extend each frame to satisfy query
;; -> only include frame in output if it cannot be extended
(define (negate operands frame-stream)
  (stream-flatmap
   (lambda (frame)
     (if (stream-null?
          (qeval (negated-query operands)
                 (singleton-stream frame)))
         (singleton-stream frame)
         the-empty-stream))
   frame-stream))

;; 'lisp-value' compound query
;; -> each frame used to instantiate variables in the pattern
;; -> lisp predicate is applied
;; -> frames where predicate == false are filtered out
;; -> error if unbound pattern variables encountered
(define (lisp-value call frame-stream)
  (stream-flatmap
   (lambda (frame)
     (if (execute
          (instantiate
           call
           frame
           (lambda (v f)
             (error
              "LISP-VALUE"
              "Unknown pat var: ~a"
              v))))
         (singleton-stream frame)
         the-empty-stream))
   frame-stream))

;; apply lisp predicate
(define (execute exp)
  (apply (eval (predicate exp) ;; eval expressions to get procedure to apply
               user-initial-environment)
         ;; don't evaluate arguments
         ;; -> already actual arguments
         ;; -> not expressions to evaluate in Lisp to produce arguments
         (args exp)))

;; special form for a query that is always satisfied
;; ignore query contents and pass through all input frames
(define (always-true ignore frame-stream)
  frame-stream)

### 4.4.4.3 Finding Assertions by Pattern Matching

In [5]:
;; returns stream of frames each extending given frame
;; by a database match of given pattern
(define (find-assertions pattern frame)
  (stream-flatmap
   (lambda (datum)
     (check-an-assertion datum pattern frame))
   ;; get stream of all assertions which should be checked for match
   (fetch-assertions pattern frame)))

;; successful match -> return one element stream of extended frame
;; failed match -> return the-empty-stream
(define (check-an-assertion
         assertion query-pat query-frame)
  (let ((match-result
         (pattern-match
          query-pat assertion query-frame)))
    (if (eq? match-result 'failed)
        the-empty-stream
        (singleton-stream match-result))))

;; successful match -> return extended frame
;; failed match -> return symbol 'failed
(define (pattern-match pat dat frame)
  (cond ((eq? frame 'failed) 'failed)
        ;; pattern and data object are the same
        ;; return frame of bindings accumulated so far
        ((equal? pat dat) frame)
        ;; pattern is a variable
        ;; extend current frame by binding variable to data
        ;; so long as consistent w/ bindings already in frame
        ((var? pat)
         (extend-if-consistent
          pat dat frame))
        ;; recursively match car of pattern against car of data
        ;; to produce a frame
        ;; -> match cdr of pattern against cdr of data in this frame
        ((and (pair? pat)
              (pair? dat))
         (pattern-match
          (cdr pat)
          (cdr dat)
          (pattern-match
           (car pat)
           (car dat)
           frame)))
        (else 'failed)))

;; extend a frame by adding a new binding
;; if consistent with existing bindings in frame
(define (extend-if-consistent var dat frame)
  (let ((binding (binding-in-frame var frame)))
    (if binding
        ;; existing binding
        ;; -> match data against value of variable in frame
        ;; if stored value contains only constants
        ;; -> match tests whether stored and new values are the same
        ;; -> if same, returns unmodified frame, else failure
        ;; recursive call will add/check bindings for vars in the pattern
        ;; -> vars could be bound to other vars
        (pattern-match
         (binding-value binding) dat frame)
        ;; no existing binding 
        ;; -> add binding of var to data
        (extend var dat frame))))

### 4.4.4.4 Rules and Unification

In [6]:
;; return stream of extension frames by applying rules from database
(define (apply-rules pattern frame)
  (stream-flatmap
   (lambda (rule)
     (apply-a-rule rule pattern frame))
   ;; get possibly applicable rules
   (fetch-rules pattern frame)))

;; try to augment argument frame by unifying rule conclusion w/ query pattern
;; success -> evaluate rule body in new frame
;; fail -> return the-empty-stream
(define (apply-a-rule rule
                      query-pattern
                      query-frame)
  ;; rename all variables in rule to have unique names
  ;; -> prevent vars for different rule applications 
  ;;    becoming confused /w each other
  (let ((clean-rule
         (rename-variables-in rule)))
    (let ((unify-result
           (unify-match query-pattern
                        (conclusion clean-rule)
                        query-frame)))
      (if (eq? unify-result 'failed)
          the-empty-stream
          (qeval (rule-body clean-rule)
                 (singleton-stream
                  unify-result))))))

;; rename all variables in a rule to be unique
;; -> associate a unique id w/ each rule application
;; -> combine id with original var names
(define (rename-variables-in rule)
  (let ((rule-application-id
         (new-rule-application-id)))
    (define (tree-walk exp)
      (cond ((var? exp)
             (make-new-variable
              exp
              rule-application-id))
            ((pair? exp)
             (cons (tree-walk (car exp))
                   (tree-walk (cdr exp))))
            (else exp)))
    (tree-walk rule)))

;; unfication successful -> return extended frame
;; unification failed -> return symbol 'failed
(define (unify-match p1 p2 frame)
  (cond ((eq? frame 'failed) 'failed)
        ((equal? p1 p2) frame)
        ;; symmetrical -> allow vars on both side of the match
        ((var? p1)
         (extend-if-possible p1 p2 frame))
        ((var? p2)
         (extend-if-possible p2 p1 frame))
        ((and (pair? p1)
              (pair? p2))
         (unify-match
          (cdr p1)
          (cdr p2)
          (unify-match
           (car p1)
           (car p2)
           frame)))
        (else 'failed)))

;; extend a frame by adding a new binding
;; if consistent in unification
(define (extend-if-possible var val frame)
  (let ((binding (binding-in-frame var frame)))
    (cond (binding
           (unify-match
            (binding-value binding) val frame))
          ;; variable trying to match is not bound and
          ;; value trying to match is itself a variable
          ((var? val)
           (let ((binding
                  (binding-in-frame
                   val
                   frame)))
             (if binding
                 ;; value variable is bound
                 ;; -> match it's value
                 (unify-match
                  var
                  (binding-value binding)
                  frame)
                 ;; both parties to match are unbound
                 ;; -> bind either to other
                 (extend var val frame))))
          ;; reject attempts to bind a variable to a pattern that includes that variable
          ;; binding a variable to itself is OK
          ;; -> handled by equal? clause in unify-match
          ((depends-on? val var frame)
           'failed)
          (else (extend var val frame)))))

;; predicate to test whether an expression proposed to be
;; the value of a pattern variable depends on that variable
;; -> relative to current frame
;; -> expression may contain occurrences of a variable that already
;;    has a value depending on the test variable
(define (depends-on? exp var frame)
  (define (tree-walk e)
    (cond ((var? e)
           (if (equal? var e)
               ;; variable is expression -> depends
               #t
               (let 
                 ((b (binding-in-frame
                      e
                      frame)))
                 (if b
                     ;; expression is bound to variable
                     ;; -> test value
                     (tree-walk
                      (binding-value b))
                     ;; expression unbound -> does not depend
                     #f))))
          ((pair? e)
           (or (tree-walk (car e))
               (tree-walk (cdr e))))
          (else #f)))
  (tree-walk exp))

### 4.4.4.5 Maintaining the Database

In [18]:
(define THE-ASSERTIONS the-empty-stream)

(define (fetch-assertions pattern frame)
  (if (use-index? pattern)
      (get-indexed-assertions pattern)
      (get-all-assertions)))

(define (get-all-assertions) THE-ASSERTIONS)

(define (get-indexed-assertions pattern)
  (get-stream (index-key-of pattern)
              'assertion-stream))

;; look up stream in table
;; return the-empty-stream if not found
(define (get-stream key1 key2)
  (let ((s (get key1 key2)))
    (if s s the-empty-stream)))

(define THE-RULES the-empty-stream)

(define (fetch-rules pattern frame)
  (if (use-index? pattern)
      (get-indexed-rules pattern)
      (get-all-rules)))

(define (get-all-rules) THE-RULES)

(define (get-indexed-rules pattern)
  (stream-append
   ;; fetch patterns where car is a constant symbol
   (get-stream (index-key-of pattern)
               'rule-stream)
   ;; fetch patterns where car is variable
   (get-stream '? 'rule-stream)))


;; add assertions and rules to database
;; used by query-driver-loop
(define (add-rule-or-assertion! assertion)
  (if (rule? assertion)
      (add-rule! assertion)
      (add-assertion! assertion)))

;; rules/assertions stored in index if appropriate
;; and in stream of all rules/assertions

(define (add-assertion! assertion)
  (store-assertion-in-index assertion)
  (let ((old-assertions THE-ASSERTIONS))
    (set! THE-ASSERTIONS
          (cons-stream assertion
                       old-assertions))
    'ok))

(define (add-rule! rule)
  (store-rule-in-index rule)
  (let ((old-rules THE-RULES))
    (set! THE-RULES
          (cons-stream rule old-rules))
    'ok))

;; store a rule/assertion
;; -> check if can be indexed
;; -> if so, store in appropriate rule/assertion stream
;; -> else no-op

(define (store-assertion-in-index assertion)
  (if (indexable? assertion)
      (let ((key (index-key-of assertion)))
        (let ((current-assertion-stream
               (get-stream
                key 'assertion-stream)))
          (put key
               'assertion-stream
               (cons-stream
                assertion
                current-assertion-stream))))))

(define (store-rule-in-index rule)
  (let ((pattern (conclusion rule)))
    (if (indexable? pattern)
        (let ((key (index-key-of pattern)))
          (let ((current-rule-stream
                 (get-stream 
                  key 'rule-stream)))
            (put key
                 'rule-stream
                 (cons-stream 
                  rule
                  current-rule-stream)))))))

;; pattern stored in table if starts w/ variable or constant symbol
(define (indexable? pat)
  (or (constant-symbol? (car pat))
      (var? (car pat))))

;; key to store a pattern
;; -> ? if pattern start w/ variabl
;; -> else constant symbol with which pattern starts
(define (index-key-of pat)
  (let ((key (car pat)))
    (if (var? key) '? key)))

;; index will be used to retrieve items
;; matching a pattern if pattern starts w/ constant symbol
(define (use-index? pat)
  (constant-symbol? (car pat)))


;; from instructors manual

(define get '())
(define put '())

(define (initialize-data-base rules-and-assertions)
  (define (deal-out r-and-a rules assertions)
    (cond ((null? r-and-a)
           (set! THE-ASSERTIONS (list->stream assertions))
           (set! THE-RULES (list->stream rules))
           (void))
          (else
           (let ((s (query-syntax-process (car r-and-a))))
             (cond ((rule? s)
                    (store-rule-in-index s)
                    (deal-out (cdr r-and-a)
                              (cons s rules)
                              assertions))
                   (else
                    (store-assertion-in-index s)
                    (deal-out (cdr r-and-a)
                              rules
                              (cons s assertions))))))))
  (let ((operation-table (make-table)))
    (set! get (operation-table 'lookup-proc))
    (set! put (operation-table 'insert-proc!)))
  (put 'and 'qeval conjoin)
  (put 'and2 'qeval conjoin2)
  (put 'or 'qeval disjoin)
  (put 'not 'qeval negate)
  (put 'lisp-value 'qeval lisp-value)
  (put 'always-true 'qeval always-true)
  (put 'unique 'qeval uniquely-asserted) ;; ex 4.75
  (deal-out rules-and-assertions '() '()))

;; from section 4.4.1
(define microshaft-database
  '((address (Bitdiddle Ben) (Slumerville (Ridge Road) 10))
    (job (Bitdiddle Ben) (computer wizard))
    (salary (Bitdiddle Ben) 60000)
    
    (address (Hacker Alyssa P) (Cambridge (Mass Ave) 78))
    (job (Hacker Alyssa P) (computer programmer))
    (salary (Hacker Alyssa P) 40000)
    (supervisor (Hacker Alyssa P) (Bitdiddle Ben))

    (address (Fect Cy D) (Cambridge (Ames Street) 3))
    (job (Fect Cy D) (computer programmer))
    (salary (Fect Cy D) 35000)
    (supervisor (Fect Cy D) (Bitdiddle Ben))

    (address (Tweakit Lem E) (Boston (Bay State Road) 22))
    (job (Tweakit Lem E) (computer technician))
    (salary (Tweakit Lem E) 25000)
    (supervisor (Tweakit Lem E) (Bitdiddle Ben))

    (address (Reasoner Louis) (Slumerville (Pine Tree Road) 80))
    (job (Reasoner Louis) (computer programmer trainee))
    (salary (Reasoner Louis) 30000)
    (supervisor (Reasoner Louis) (Hacker Alyssa P))

    (supervisor (Bitdiddle Ben) (Warbucks Oliver))

    (address (Warbucks Oliver) (Swellesley (Top Heap Road)))
    (job (Warbucks Oliver) (administration big wheel))
    (salary (Warbucks Oliver) 150000)

    (address (Scrooge Eben) (Weston (Shady Lane) 10))
    (job (Scrooge Eben) (accounting chief accountant))
    (salary (Scrooge Eben) 75000)
    (supervisor (Scrooge Eben) (Warbucks Oliver))

    (address (Cratchet Robert) (Allston (N Harvard Street) 16))
    (job (Cratchet Robert) (accounting scrivener))
    (salary (Cratchet Robert) 18000)
    (supervisor (Cratchet Robert) (Scrooge Eben))

    (address (Aull DeWitt) (Slumerville (Onion Square) 5))
    (job (Aull DeWitt) (administration secretary))
    (salary (Aull DeWitt) 25000)
    (supervisor (Aull DeWitt) (Warbucks Oliver))

    (can-do-job (computer wizard) (computer programmer))
    (can-do-job (computer wizard) (computer technician))

    (can-do-job (computer programmer)
                (computer programmer trainee))

    (can-do-job (administration secretary)
                (administration big wheel))

    (rule (lives-near ?person-1 ?person-2)
          (and (address ?person-1 (?town . ?rest-1))
               (address ?person-2 (?town . ?rest-2))
               (not (same ?person-1 ?person-2))))

    (rule (same ?x ?x))

    (rule (wheel ?person)
          (and (supervisor ?middle-manager ?person)
               (supervisor ?x ?middle-manager)))

    (rule (outranked-by ?staff-person ?boss)
          (or (supervisor ?staff-person ?boss)
              (and (supervisor ?staff-person ?middle-manager)
                   (outranked-by ?middle-manager ?boss))))))

;; reinit the database using microshaft-database
;; call from Scheme not driver loop
(define (setup-database)
  (initialize-data-base microshaft-database))

(define (make-database rules-and-assertions)
  (initialize-data-base rules-and-assertions))

(setup-database)

## 4.70

The second argument to `cons-stream` is lazy and is therefore not evaluated when `const-stream` is called. The `let` is required to force the evaluation of `THE-ASSERTIONS`, assigning it's value to the variable `old-assertions`. In the definition in the question, `THE-ASSERTIONS` in `cons-stream` would point to itself and no changes would be made:

```scheme
(define (add-assertion! assertion)
  (store-assertion-in-index assertion)
  (set! THE-ASSERTIONS
        (cons-stream assertion 
                     THE-ASSERTIONS)) ;; not evaluated -> set! has no effect
  'ok)
```

### 4.4.4.6 Stream Operations

In [8]:
;; append delayed stream to a stream
(define (stream-append-delayed s1 delayed-s2)
  (if (stream-null? s1)
      (force delayed-s2)
      (cons-stream
       (stream-car s1)
       (stream-append-delayed (stream-cdr s1)
                              delayed-s2))))

;; interleave a delayed stream with a stream
(define (interleave-delayed s1 delayed-s2)
  (if (stream-null? s1)
      (force delayed-s2)
      (cons-stream
       (stream-car s1)
       (interleave-delayed
        (force delayed-s2)
        (delay (stream-cdr s1))))))

;; map a procedure over a stream of frames
;; combine resulting stream of frames
(define (stream-flatmap proc s)
  (flatten-stream (stream-map proc s)))

(define (flatten-stream stream)
  (if (stream-null? stream)
      the-empty-stream
      (interleave-delayed
       (stream-car stream)
       (delay (flatten-stream
               (stream-cdr stream))))))

;; generate single-element stream
(define (singleton-stream x)
  (cons-stream x the-empty-stream))

### 4.4.4.7 Query Syntax Procedures

In [9]:
;; specify that a special form is identified by symbol in its car
(define (type exp)
  (if (pair? exp)
      (car exp)
      (error "TYPE"
             "unknown expression ~a"
             exp)))
(define (contents exp)
  (if (pair? exp)
      (cdr exp)
      (error "CONTENTS"
             "unknown expression ~a"
             exp)))

;; specify that rules and assertions are added to database
;; by expressions of form (assert! <rule-or-assertion>)
(define (assertion-to-be-added? exp)
  (eq? (type exp) 'assert!))
(define (add-assertion-body exp)
  (car (contents exp)))

;; syntax definitions for and, or, not and lisp-value special forms
(define (empty-conjunction? exps) (null? exps))
(define (first-conjunct exps) (car exps))
(define (rest-conjuncts exps) (cdr exps))
(define (empty-disjunction? exps) (null? exps))
(define (first-disjunct exps) (car exps))
(define (rest-disjuncts exps) (cdr exps))
(define (negated-query exps) (car exps))
(define (predicate exps) (car exps))
(define (args exps) (cdr exps))

;; rule syntax
(define (rule? statement)
  (tagged-list? statement 'rule))
(define (conclusion rule)
  (cadr rule))
(define (rule-body rule)
  (if (null? (cddr rule))
      '(always-true)
      (caddr rule)))

;; transform pattern variables of form ?symbol into internal format (? symbol)
;; -> (job ?x ?y) -> (job (? x) (? y))
;; increase efficiency when determining if expression is a pattern variable
;; -> just check car of expression instead of extracting characters from symbol
(define (query-syntax-process exp)
  (map-over-symbols expand-question-mark exp))

(define (map-over-symbols proc exp)
  (cond ((pair? exp)
         (cons (map-over-symbols
                proc
                (car exp))
               (map-over-symbols
                proc
                (cdr exp))))
        ((symbol? exp) (proc exp))
        (else exp)))

;; transform variable from input format to internal format
;; -> ?x -> (? x)
(define (expand-question-mark symbol)
  (let ((chars (symbol->string symbol)))
    (if (string=? (substring chars 0 1) "?")
        (list '? (string->symbol
                  (substring
                   chars
                   1
                   (string-length chars))))
        symbol)))

;; transform variable from internal format back to input format
;; ->  (? x) -> ?x 
(define (contract-question-mark variable)
  (string->symbol
   (string-append "?"
                  (if (number? (cadr variable))
                      (string-append
                       (symbol->string (caddr variable))
                      "-"
                       (number->string (cadr variable)))
                      (symbol->string (cadr variable))))))

(define (var? exp)
  (tagged-list? exp '?))
(define (constant-symbol? exp)
  (symbol? exp))

;; generate unique identifier for variable renaming
(define rule-counter 0)

(define (new-rule-application-id)
  (set! rule-counter (+ 1 rule-counter))
  rule-counter)

(define (make-new-variable
         var rule-application-id)
  (cons '? (cons rule-application-id
                 (cdr var))))


### 4.4.4.8 Frames and Bindings

In [10]:
;; frames = list of bindings
;; bindings = (variable value) pairs

(define (make-binding variable value)
  (cons variable value))

(define (binding-variable binding)
  (car binding))

(define (binding-value binding)
  (cdr binding))

(define (binding-in-frame variable frame)
  (assoc variable frame))

(define (extend variable value frame)
  (cons (make-binding variable value) frame))

#### Helpers

Additional helpers for testing/running queries without the driver loop

In [11]:
(define (interpret-query query)
  (let ((q (query-syntax-process query)))
    (cond ((assertion-to-be-added? q)
           (add-rule-or-assertion! (add-assertion-body q))
           (newline)
           (display "Assertion added to data base."))
          (else
           (newline)
           (display-stream
            (stream-map
             (lambda (frame)
               (instantiate q
                            frame
                            (lambda (v f)
                              (contract-question-mark v))))
             (qeval q (singleton-stream '()))))))))

## 4.71

Using explicit `delay` operations allows query results to be output even if a set of rules leads to an infinite loop. For example, given an infinitely recursive rule such as:

```scheme
;; infinitely recursive rule
(assert! (rule (person ?x) (person ?x)))

(assert! (person (Bitdiddle Ben)))
(assert! (person (Hacker P Alyssa)))
(assert! (person (Reasoner Louis)))
```
Evaluating the rule using the explicit `delay`'s in the text will give an infinite result set:

```scheme
(person ?who)
;; (person (Reasoner Louis))
;; (person (Hacker P Alyssa))
;; (person (Bitdiddle Ben))
;; (person (Reasoner Louis))
;; (person (Hacker P Alyssa))
;; (person (Bitdiddle Ben))
...
```

Without the explicit `delay`'s however, no results will ever be returned as the evaluator will attempt to find the entire set before returning any values:

```scheme
(person ?who)
;; -> will error with a stack overflow due to infinite recursion
```

Using explicit `delay`'s at least allows results to be returned instead of crashing the program. However, it is still not ideal as the program will now be stuck returning values infinitely for the query.

## 4.72

`disjoin` and `stream-flatmap` interleave the streams in order to provide useful results when infinite streams are used. Without interleaving, the first of the stream arguments will be exhausted before any results from the second stream are included. If this first stream is infinite, then the second stream will never be reached.

## 4.73

`flatten-stream` uses an explicit `delay` for the same reasons as in exercise 4.71. If the stream were infinite, `flatten-stream` would cause a stack overflow without the `delay` as the entire `stream-cdr` will be evaluated:

```scheme
(define (flatten-stream stream)
  (if (stream-null? stream)
      the-empty-stream
      (interleave (stream-car stream)
                  (flatten-stream 
                   ;; stack overflow if infinite
                   (stream-cdr stream)))))

```

## 4.74

1.

In [12]:
(define (simple-stream-flatmap proc s)
  (simple-flatten (stream-map proc s)))

;; stream is always a singleton or the empty stream
(define (simple-flatten stream)
  (stream-map
   stream-car
   ;; filter out empty streams
   (stream-filter
    (lambda (s)
      (not (stream-null? s)))
    stream)))

2. No, provided Alyssa's assertions regarding the argument to `simple-flatten` always being either a singleton or an empty stream the behaviour will be the same as with the implementation in the text.

Original implementation:

```scheme
(define (flatten-stream stream)
  (if (stream-null? stream)
      the-empty-stream ;; a
      (interleave (stream-car stream) ;; b
                  (flatten-stream 
                   (stream-cdr stream)))))
```

a. If `flatten-stream` receives an empty stream as input, it will return an empty stream.
b. If `flatten-stream` receives a singleton stream as input it will return the result of calling `interleave` with the `car` of the stream as it's first argument and the result of a recursive call to `flatten-stream` on the `cdr` of the stream as it's second. The `car` will be the single value of the stream, the recursive call to `flatten-stream` will receive an empty stream as input (`cdr` of a singleton stream) and will therefore return an empty stream as described in `a`. Producing a stream of the values in the singleton stream.

## 4.75

In [19]:
(define (singleton-stream? stream)
  (and (not (stream-null? stream))
       (stream-null? (stream-cdr stream))))

;; installed in initialize-database under 'unique key
(define (uniquely-asserted operands frame-stream)
  (stream-flatmap
   (lambda (frame)
     ;; find stream of all extensions satisfying query
     (let ((result (qeval (car operands)
                          (singleton-stream frame))))
       ;; return result stream only if unique (single element)
       (if (singleton-stream? result)
           result 
           the-empty-stream)))
   frame-stream))

In [20]:
(interpret-query '(unique (job ?x (computer wizard))))



(unique (job (Bitdiddle Ben) (computer wizard)))

done

In [21]:
(interpret-query '(unique (job ?x (computer programmer))))

done

In [22]:
(interpret-query '(and (job ?x ?j) 
                       (unique (job ?anyone ?j))))



(and (job (Aull DeWitt) (administration secretary)) (unique (job (Aull DeWitt) (administration secretary))))
(and (job (Cratchet Robert) (accounting scrivener)) (unique (job (Cratchet Robert) (accounting scrivener))))
(and (job (Scrooge Eben) (accounting chief accountant)) (unique (job (Scrooge Eben) (accounting chief accountant))))
(and (job (Warbucks Oliver) (administration big wheel)) (unique (job (Warbucks Oliver) (administration big wheel))))
(and (job (Reasoner Louis) (computer programmer trainee)) (unique (job (Reasoner Louis) (computer programmer trainee))))
(and (job (Tweakit Lem E) (computer technician)) (unique (job (Tweakit Lem E) (computer technician))))
(and (job (Bitdiddle Ben) (computer wizard)) (unique (job (Bitdiddle Ben) (computer wizard))))

done

## 4.76

In [25]:
;; installed in setup-database above as and2
(define (conjoin2 conjuncts frame-stream)
  (if (empty-conjunction? conjuncts)
      frame-stream
      (merge-streams-of-frames
       (qeval (first-conjunct conjuncts)
              frame-stream)
       (conjoin2 (rest-conjuncts conjuncts)
                frame-stream))))

;; For each binding in each frame of s1, check if consistent
;; with corresponding frame in s2. If so -> merge frame
(define (merge-streams-of-frames s1 s2)
 (define attempted-merges
   (stream-map
    (lambda (f1)
      (stream-map
       (lambda (f2)
         (merge-frames-if-consistent f1 f2))
       s2))
    s1)) 
  (define successful-merges
    (stream-filter
     (lambda (f)
       (not (eq? f 'failed)))
     attempted-merges))
  (flatten-stream successful-merges))
       
      
;; for each binding in f1, check if consistent w/f2 (extend-if-possible)
;; return 'failed if any binding is inconsistent
(define (merge-frames-if-consistent f1 f2)
  (if (null? f1)
      f2
      (let* ((binding (first-binding f1))
             (extension (extend-if-possible
                         (binding-value binding)
                         (binding-variable binding)
                         f2)))
        (if (eq? extension 'failed)
            'failed
            (merge-frames-if-consistent
             (rest-bindings f1)
             extension)))))
      
(define (first-binding frame)
  (car frame))
      
(define (rest-bindings frame)
  (cdr frame))
(setup-database)

(interpret-query
  '(and2 
    (job ?x ?j) 
    (supervisor ?x (Bitdiddle Ben))))



(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor ?x (Bitdiddle Ben)))
(and2 (job ?x ?j) (supervisor

done

# 4.4 Logic Programming

## 4.55

```scheme
;; 1
(supervisor ?x (Bitdiddle Ben))

;; 2
(job ?x (accounting . ?type))

;; 3
(address ?x (Sumerville . ?y))
```

## 4.56

```scheme
;; 1
(and (supervisor ?person (Bitdiddle Ben))
     (address ?person . ?street))
;; 2
(and (salary (Bitdiddle Ben) ?x)
     (salary ?person ?amount)
     (lisp-value < ?amount ?x))

;; 3
(and (supervisor ?person ?supervisor)
     (job ?supervisor ?job)
     (not (job ?supervisor (computer . ?type))))
```

## 4.57

```scheme
(rule (can-replace ?person1 ?person2)
      (and
       (or
        ;; person1 does same job as person2
        (and (job ?person1 ?job) 
             (job ?person2 ?job))
        ;; OR someone who does person1's job can also do person2's job
        (and (job ?person1 ?job1)
             (job ?person2 ?job2)
             (can-do-job ?j1 ?j2)))
       ;; person1 and person2 are not the same person
       (not (same ?person1 ?person2))))

;; 1
(can-replace ?person (Fect Cy D))

;; 2 

(and (can-replace ?person1 ?person2)
     (salary ?person1 ?salary1)
     (salary? ?person2 ?salary2)
     (lisp-value > salary2 salary1))
```

## 4.58

```scheme
(rule (big-shot ?person ?division)
      (and 
       ;; person works in given division
       (job ?person (?division . ?type))
       (supervisor ?person ?supervisor)
       (job ?supervisor (?sup-division . ?sup-type))
       ;; persons superervisor does not work in given divistion
       (not (same ?division ?sup-division))))
```

## 4.59

```scheme
;; 1
(meeting ?division (Friday ?time))

;; 2
(rule (meeting-time ?person ?day-and-time)
      (or (meeting whole-company ?day-and-time)
          (and
           (job ?person (?division . ?type))
           (meeting ?division ?day-and-time))))

;; 3
(meeting-time (Hacker Alyssa P) (Wednesday ?time))
```

## 4.60

The `lives-near` rule is commutative, so the following two queries will both return the same *values* but in the opposite order:

```scheme
(lives-near ?person-1 ?person-2)
(lives-near ?person-2 ?person-1)
```

This is because `lives-near` is implemented using an `and` expression, which is in itself commutative:

```scheme
(rule (lives-near ?person-1 ?person-2)
      ;; arguments to and could be in any order
      (and (address ?person-1 
                    (?town . ?rest-1))
           (address ?person-2 
                    (?town . ?rest-2))
           (not (same ?person-1 ?person-2))))
```

This poses no restrictions on the values that `?person-1` and `?person-2` can take, so every *combination* that the query succeeds for is returned.

To produce a list of *unique pairs* of people who live near each other, a futher restriction can be added to the `lives-near` rule that enforces an **ordering** of the pairs. For example, enforcing alphabetical ordering of `person-1` and `person-2` in the pair will mean only one combination of the two people is returned. For example, `(Hacker Alyssa P) (Fect Cy D))` is alphabetically *less* than `((Fect Cy D) (Hacker Alyssa P))`.

In [37]:
;; convert a list of symbols into a single string
(define (symbol-list->string lst)
  (accumulate string-append "" (map symbol->string lst)))

;; compare two lists of symbols to determine if
;; lst1 is alphabetically less than lst2
(define (symbol-list<? lst1 lst2)
  (string<? (symbol-list->string lst1)
            (symbol-list->string lst2)))

(symbol-list<? '(Hacker Alyssa P) '(Fect Cy D))

#f

In [38]:
(symbol-list<?  '(Fect Cy D) '(Hacker Alyssa P))

#t

```scheme
(rule (lives-near ?person-1 ?person-2)
      (and (address ?person-1 
                    (?town . ?rest-1))
           (address ?person-2 
                    (?town . ?rest-2))
           (not (same ?person-1 ?person-2))
           (lisp-value symbol-list>? ?person-1 ?person-1)))
```

## 4.61

```scheme
(rule (?x next-to ?y in (?x ?y . ?u)))
(rule (?x next-to ?y in (?v . ?z))
      (?x next-to ?y in ?z))

(?x next-to ?y in (1 (2 3) 4))
;; ((2 3) next-to 4 in (1 (2 3) 4))
;; (1 next-to (2 3) in (1 (2 3) 4))

(?x next-to 1 in (2 1 3 1))
;; (2 next-to 1 in (2 1 3 1))
;; (3 next-to 1 in (2 1 3 1))
```

## 4.62 - TODO Test after implemented query evaluator

#### Expected results from examples:

```scheme
(last-pair (3) ?x)
;; (last-pair (3) (3))

(last-pair (1 2 3) ?x)
;; (last-pair (1 2 3) (3))

(last-pair (2 ?x) (3))
;; (last-pair (2 3) (3))

;; not possible -> infinite possible solutions:
(last-pair ?x (3))
;; (last-pair (3) (3))
;; (last-pair (1 3) (3))
;; (last-pair (2 3) (3))
;; (last-pair (3 3) (3))
;; ...
```

#### `last-pair` rules:

- For a single element list `x`, `last-pair` is `x`
- For any `x`, `y`, `z`, `last-pair` of `(cons x y)` is `z` *if* `last-pair` of `y` is `z`

```scheme
(rule (last-pair (?x) ?x))
(rule (last-pair (?x . ?y) ?z)
      (last-pair ?y (?z)))
```

## 4.63 TODO Test after implemented query evaluator

```scheme
(rule (grandson ?g ?s)
      (and (son ?f ?s)
           (son ?g ?f)))
(rule (son ?w ?s)
      (and (wife ?m ?w)
           (son ?w ?s)))
```

## 4.64

Louis' `outranked-by` rule

```scheme
(rule (outranked-by ?staff-person ?boss)
      (or (supervisor ?staff-person ?boss)
          (and (outranked-by ?middle-manager
                             ?boss)
               (supervisor ?staff-person
                           ?middle-manager))))
```

Method for applying a rule described in 4.4.2:
1. Unify the query with the conclusion of the rule to form, if successful, an extension of the original frame.
2. Relative to the *extended* frame, evaluate the query formed by the body of the rule.

Unify the query with the conclusion of the rule:
```scheme
;; query
(outranked-by (Bitdiddle Ben) ?who)

;; conclusion of rule
(outranked-by ?staff-person ?boss)
```

This `succeeds`, extending the original frame with a binding from `?staff-person` -> `(Bitdiddle Ben)` and from `?boss` -> `?who`:

```scheme
;; body of rule
(or (supervisor ?staff-person ?boss)
    (and (outranked-by ?middle-manager
                       ?boss)
         (supervisor ?staff-person
                     ?middle-manager)))

;; with the extended binding from step 1 becomes
(or (supervisor (Bitdiddle Ben) ?who)
    (and (outranked-by ?middle-manager
                       ?who)
         (supervisor (Bitdiddle Ben)
                     ?middle-manager)))
```

Evaluating the `outranked-by` query within the `and` expression:

```scheme
;; query
(outranked-by ?middle-manager ?who)

;; conclusion of rule
(outranked-by ?staff-person ?boss)
```

Again, this succeeds, extending the original frame with a binding from `?staff-person` -> `?middle-manager` and from `?boss` -> `?who`.

```scheme
;; body of rule within extended binding
(or (supervisor ?middle-manager ?who)
    (and (outranked-by ?middle-manager
                       ?who)
         (supervisor ?middle-manager
                     ?middle-manager)))
```

The `outranked-by` query within the `and` expression is evaluated again, binding `?staff-person` -> `?middle-manager` and `?boss` -> `?who`, repeating this infinitely.

The original definition of `outranked-by` from 4.4.1 doesn't cause an infinite loop because the sub-queries within the `and` expression are in opposite order. The `supervisor` query is evaluated first which creates bindings for `?staff-person` and `?middle-manager` which restrict the search space for the recursive `outranked-by` query.

## 4.65

`wheel` rule from 4.4.1:
```scheme
(rule (wheel ?person)
      (and (supervisor ?middle-manager
                       ?person)
           (supervisor ?x ?middle-manager)))
```

The query `(wheel ?who)` results in Oliver Warbucks being listed four times because the database contains entries corresponding to four middle managers who have Oliver Warbucks as a manager. The `wheel` query does not put any restriction on uniqueness of results and will just return all matches found in the database. Using the database defined in 4.4.1, Oliver Warbucks is the manager of the following middle managers:
- Ben Bitdiddle 
- Eben Scrooge

Ben Bitdiddle, however, has entries which describe 3 different people for whom he is the middle manager:
- Lem E Tweakit
- Cy D Fect
- Alyssa P Hacker

There is one entry for whom Ben Bitdiddle is the `wheel`:
- Alyssa P Hacker
    - Middle manager of Louis Reasoner

Each of these will return Oliver Warbucks as being the `wheel`, as he is the manager of Ben Bitdiddle.

## 4.66

Ben realised that query results can contain duplicates which would produce incorrect values for aggregation functions. For example, computing the `sum` of the salaries of all the wheels (4.65) in the business:

```scheme
(sum ?amount
     (and (wheel ?who)
          (salary ?who ?amount)))
```
As shown in 4.65, the `wheel` query will return Oliver Warbucks four times and as such the `sum` result will include his salary four times.

To fix this, Ben will need to enforce a uniqueness constraint on the results from the `query pattern` passed to the `accumulation-function`. This could be implemented at the evaluator level, by tracking intermediate results from `query pattern` in association with the sub-query that produced the result and the current frame.

For example, the result `(wheel (Warbucks Oliver))` returned from the `wheel` sub-query would be stored under a `wheel` key for the current environment. A subsequent result from `(wheel ?who)` that would return `(wheel (Warbucks Oliver))` would be found to already be present and then not included in the returned result set. Storing values in association with *both* the current frame and sub-query is important to adhere to the query evaluator's unification rules - the 'parallel' evaluation of `or` and the serial evaluation of `and` is handled by capturing the current frame for the sub-query.

## 4.67 TODO AFTER IMPLEMENTING EVALUATOR



## 4.68 TODO AFTER IMPLEMENTING EVALUATOR

## 4.69 TODO AFTER IMPLEMENTING EVALUATOR